In [ ]:
import cv2
import matplotlib.pyplot as plot
import numpy as np

camera_img = cv2.imread("X.png")
map_img = cv2.imread("Xmap.png")

gray1 = cv2.cvtColor(camera_img, cv2.COLOR_BGRA2GRAY)
gray2 = cv2.cvtColor(map_img, cv2.COLOR_BGRA2GRAY)


In [ ]:
# Use orb algorithm to find features, have not have success with this
orb = cv2.ORB_create()

kp1, des1 = orb.detectAndCompute(gray1, None)
kp2, des2 = orb.detectAndCompute(gray2, None)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1, des2)

matches = sorted(matches, key=lambda x: x.distance)


In [ ]:
img_matches = cv2.drawMatches(camera_img, kp1, map_img, kp2, matches[:10], None, 
                              flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

plot.imshow(img_matches)
plot.show()

In [ ]:
pt1 = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
pt2 = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

H, mask = cv2.findHomography(pt1, pt2, cv2.RANSAC, 5.0)

img = cv2.imread('archive/imgs/checkerboard.png')
height, width, _ = img.shape
img_aligned = cv2.warpPerspective(img, H, (width, height))

plot.imshow(img_aligned)
plot.show()


In [ ]:
# Successfully warps camera view of intersection to map view
x=0
y=0
# Define source and destination points
srcPoints = np.array([
    [55, 159],
    [33, 137], 
    [237, 106], 
    [276, 118], 
], dtype=np.float32)
dstPoints = np.array([
    [709 + x, 257 + y],
    [698 + x, 91 + y], 
    [256 + x, 94 + y], 
    [256 + x, 246 + y], 
], dtype=np.float32)

# Compute homography matrix
H, mask = cv2.findHomography(srcPoints, dstPoints, cv2.RANSAC, 5.0)
height, width, _ = map_img.shape

img_aligned = cv2.warpPerspective(cv2.imread("FRAME.png"), H, (width, height))

plot.imshow(img_aligned)
pass

In [ ]:
src = [55, 159]
# Internal homography matrix math warping the pixels from camera to map view
dest = [(src[0] * H[0][0] + src[1] * H[0][1] + H[0][2])/(src[0] * H[2][0] + src[1] * H[2][1] + H[2][2]),
        (src[0] * H[1][0] + src[1] * H[1][1] + H[1][2])/(src[0] * H[2][0] + src[1] * H[2][1] + H[2][2])]

dest

In [ ]:
cv2.imwrite('matches.png', img_matches)

In [ ]:
cv2.imwrite('aligned.png', img_aligned)

In [ ]:
img = cv2.imread('exit11.png')
p1 = [118, 147]

height, width, _ = map_img.shape
# cv2.circle(m, center=p1, radius=20, color=(255, 0, 0), thickness=-1)
img2 = cv2.warpPerspective(camera_img, H, (width*4, height*4))
plot.imshow(img2)
plot.show()

In [ ]:
match = matches[6]

print(match)
print(match.distance)
print(match.imgIdx)
print(match.queryIdx)
print(match.trainIdx)
print(len(kp1))

img_points1 = cv2.drawKeypoints(gray1, kp1, None)

plot.imshow(img_points1)
plot.show()

In [ ]:
import subprocess
import os

reader = cv2.VideoCapture("z.mp4")
total_frames = int(reader.get(cv2.CAP_PROP_FRAME_COUNT))
fps = reader.get(cv2.CAP_PROP_FPS)

shape = (int(reader.get(cv2.CAP_PROP_FRAME_WIDTH)), int(reader.get(cv2.CAP_PROP_FRAME_HEIGHT)))

output_file = "y.mp4"
intermediate_filemp4 = "__inter__.mp4"
writer = cv2.VideoWriter(intermediate_filemp4, cv2.VideoWriter_fourcc(*"mp4v"), fps, shape)

for frameid in range(total_frames):
    ret, frame = reader.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    kp, des = orb.detectAndCompute(gray, None)

    points = cv2.drawKeypoints(frame, kp, None)
    writer.write(points)
writer.release()

if os.path.exists(output_file):
    os.remove(output_file)
subprocess.run(["ffmpeg", "-i", intermediate_filemp4, output_file])
os.remove(intermediate_filemp4)
